In [1]:
import scanpy as sc
from lataq.models import EMBEDCVAE
%load_ext autoreload
%autoreload 2

In [3]:
adata_meyer = sc.read(
    '/storage/groups/ml01/workspace/'
    'hlca_lisa.sikkema_malte.luecken/'
    'HLCA_reproducibility/data/HLCA_extended/'
    'extension_datasets/raw/Meyer/Meyer_2021_raw.h5ad'
)

In [5]:
adata = sc.read(
    '/storage/groups/ml01/workspace/'
    'hlca_lisa.sikkema_malte.luecken/'
    'HLCA_reproducibility/data/HLCA_extended/'
    'HLCA_core_plus_single_study/'
)

OSError: Unable to open file (unable to open file: name = '/storage/groups/ml01/workspace/hlca_lisa.sikkema_malte.luecken/HLCA_reproducibility/data/HLCA_extended/HLCA_core_plus_single_study.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [4]:
adata_meyer.obs.groupby(['ann_level_5_clean']).size()

ann_level_5_clean
CD4 T cells MAIT       1047
CD4 T cells naive      2543
CD4T cells TRM         1588
CD8 T cells GZMK+      1277
CD8 T cells ME         1753
CD8 T cells TRM        1044
Mature B cells          902
NK_ITGAD+               789
NK_XCL1+                802
Naive B cells           358
None                 116468
Plasmablasts             57
dtype: int64

In [12]:
adata = sc.read(
    '/storage/groups/ml01/workspace/'
    'hlca_lisa.sikkema_malte.luecken/'
    'github_repo/data/HLCA_extended/'
    'extension_datasets/ready/subsetted/lambrechts_sub.h5ad'
)

In [13]:
adata.var_names

Index(['FGR', 'CFH', 'HS3ST1', 'TMEM176A', 'TFPI', 'HSPB6', 'PDK4', 'ZMYND10',
       'WDR54', 'ITGAL',
       ...
       'AP000769.1', 'AC133644.2', 'TRG-AS1', 'CH17-262H11.1', 'RP1-34B20.21',
       'TRBV6-2', 'PRB4', 'PRB1', 'MUC7', 'CRNN'],
      dtype='object', length=2000)

In [14]:
adata_meyer.var['gene_symbols'] = adata_meyer.var['gene_symbols'].astype('str')
adata_meyer.var.set_index('gene_symbols', inplace=True)

In [15]:
adata_meyer.var_names_make_unique()

In [16]:
adata_meyer = adata_meyer[:, adata.var_names]
adata_meyer.var_names_make_unique()

In [25]:
adata_conc = adata_meyer.concatenate([adata_meyer, adata_meyer])

In [30]:
import time

start = time.time()
lataq_model = EMBEDCVAE(
    adata=adata_conc,
    condition_key='subject_ID',
    cell_type_keys=['ann_level_5'],
    hidden_layer_sizes=[128]*4,
    latent_dim=50,
)
lataq_model.train(
    n_epochs=1,
    pretraining_epochs=1,
    weight_decay=0,
    use_stratified_sampling=False,
)
print(time.time()-start)

Embedding dictionary:
 	Num conditions: 7
 	Embedding dim: 10
Encoder Architecture:
	Input Layer in, out and cond:ERROR! Session/line number was not unique in database. History logging moved to new session 3855
 2000 128 0
	Hidden Layer 1 in/out: 128 128
	Hidden Layer 2 in/out: 128 128
	Hidden Layer 3 in/out: 128 128
	Mean/Var Layer in/out: 128 50
Decoder Architecture:
	First Layer in, out and cond:  50 128 10
	Hidden Layer 1 in/out: 128 128
	Hidden Layer 2 in/out: 128 128
	Hidden Layer 3 in/out: 128 128
	Output Layer in/out:  128 2000 

loaders init
loaders init done
0.0005297660827636719
 |████████████████████| 100.0%  - val_loss: 536.9392161590 - val_trvae_loss: 536.9392161590
65.83386540412903


In [27]:
adata_conc

AnnData object with n_obs × n_vars = 385884 × 2000
    obs: 'sample', 'original_celltype_ann', 'study_long', 'study', 'last_author_PI', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'ann_level_1_clean', 'ann_level_2_clean', 'ann_level_3_clean', 'ann_level_4_clean', 'ann_level_5_clean', 'subject_ID', 'subject_ID_as_published', 'Processing_site', 'pre_or_postnatal', 'age_in_years', 'age_range', 'sex', 'ethnicity', 'mixed_ethnicity', 'smoking_status', 'smoking_history', 'BMI', 'known_lung_disease', 'condition', 'subject_type', 'cause_of_death', 'sample_type', 'anatomical_region_coarse', 'anatomical_region_detailed', 'tissue_dissociation_protocol', 'cells_or_nuclei', 'single_cell_platform', "3'_or_5'", 'enrichment', 'sequencing_platform', 'reference_genome_coarse', 'ensembl_release_reference_genome', 'cell_ranger_version', 'disease_status', 'fresh_or_frozen', 'cultured', 'cell_viability_%', 'comments', 'dataset', 'trvae_size_factor

In [19]:
start = time.time()
lataq_model = EMBEDCVAE(
    adata=adata_conc,
    condition_key='subject_ID',
    hidden_layer_sizes=[128]*4,
    latent_dim=50,
)
lataq_model.train(
    n_epochs=1,
    pretraining_epochs=1,
    weight_decay=0,
    train_frac=1,
)
print(time.time()-start)

Embedding dictionary:
 	Num conditions: 7
 	Embedding dim: 10
Encoder Architecture:
	Input Layer in, out and cond: 2000 128 0
	Hidden Layer 1 in/out: 128 128
	Hidden Layer 2 in/out: 128 128
	Hidden Layer 3 in/out: 128 128
	Mean/Var Layer in/out: 128 50
Decoder Architecture:
	First Layer in, out and cond:  50 128 10
	Hidden Layer 1 in/out: 128 128
	Hidden Layer 2 in/out: 128 128
	Hidden Layer 3 in/out: 128 128
	Output Layer in/out:  128 2000 

loaders init
loaders init done
113.14403486251831
 |████████████████████| 100.0% 
133.30422449111938


In [20]:
import scarches as sca

In [21]:
start = time.time()
trvae = sca.models.TRVAE(
    adata=adata_meyer,
    condition_key='subject_ID',
)
trvae.train(
    n_epochs=1,
)
print(time.time()-start)


INITIALIZING NEW NETWORK..............
Encoder Architecture:
	Input Layer in, out and cond: 2000 256 7
	Hidden Layer 1 in/out: 256 64
	Mean/Var Layer in/out: 64 10
Decoder Architecture:
	First Layer in, out and cond:  10 64 7
	Hidden Layer 1 in/out: 64 256
	Output Layer in/out:  256 2000 

 |████████████████████| 100.0%  - val_loss: 588.4204047175 - val_recon_loss: 533.3745035606 - val_kl_loss: 13.2457308911 - val_mmd_loss: 41.8001724659
Saving best state of network...
Best State was in Epoch 0
62.29477262496948
